# MIMIC-IV Clinical Notes Exploration

This notebook explores the structure and content of MIMIC-IV clinical notes to inform model selection for text processing. We analyze:
- **Discharge summaries** and **Radiology reports**
- Text length distributions to determine if we need standard BERT (512 tokens) or Longformer/ModernBERT (8k+ tokens)
- Linkage between text files and metadata (_detail) files

**Context**: Preparing unstructured clinical text for an RL agent (P-CAFE).

## Section 1: Setup & Imports

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style for better readability
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

## Section 2: Define File Paths

Update these paths to match your MIMIC-IV data location. The files should be in the `note` module of MIMIC-IV.

In [ ]:
# Define file paths - Update these to match your data location
# Standard MIMIC-IV structure: physionet.org/files/mimiciv/3.1/note/

# Adjust this base path to your MIMIC-IV data directory
base_path = 'C:\\Users\\Eli\\Data\\physionet.org\\files\\mimic-iv-note\\2.2\\note\\'

# Clinical text files
discharge_file = base_path + 'discharge.csv.gz'
radiology_file = base_path + 'radiology.csv.gz'

# Metadata files
discharge_detail_file = base_path + 'discharge_detail.csv.gz'
radiology_detail_file = base_path + 'radiology_detail.csv.gz'

print("File paths defined.")
print(f"Discharge: {discharge_file}")
print(f"Discharge Detail: {discharge_detail_file}")
print(f"Radiology: {radiology_file}")
print(f"Radiology Detail: {radiology_detail_file}")

## Section 3: Load Data (Optimized)

Since note files contain large text fields, we load only the first 1,000 rows of each file to inspect structure without memory issues.

In [ ]:
# Load discharge notes (first 1000 rows)
print("Loading discharge notes (first 1000 rows)...")
discharge_df = pd.read_csv(discharge_file, nrows=1000, compression='gzip')
print(f"Discharge notes loaded: {len(discharge_df)} rows")
print(f"Columns: {list(discharge_df.columns)}")
print(f"Data types:\n{discharge_df.dtypes}")
print()

In [ ]:
# Load discharge detail (first 1000 rows)
print("Loading discharge detail (first 1000 rows)...")
discharge_detail_df = pd.read_csv(discharge_detail_file, nrows=1000, compression='gzip')
print(f"Discharge detail loaded: {len(discharge_detail_df)} rows")
print(f"Columns: {list(discharge_detail_df.columns)}")
print(f"Data types:\n{discharge_detail_df.dtypes}")
print()

In [ ]:
# Load radiology notes (first 1000 rows)
print("Loading radiology notes (first 1000 rows)...")
radiology_df = pd.read_csv(radiology_file, nrows=1000, compression='gzip')
print(f"Radiology notes loaded: {len(radiology_df)} rows")
print(f"Columns: {list(radiology_df.columns)}")
print(f"Data types:\n{radiology_df.dtypes}")
print()

In [ ]:
# Load radiology detail (first 1000 rows)
print("Loading radiology detail (first 1000 rows)...")
radiology_detail_df = pd.read_csv(radiology_detail_file, nrows=1000, compression='gzip')
print(f"Radiology detail loaded: {len(radiology_detail_df)} rows")
print(f"Columns: {list(radiology_detail_df.columns)}")
print(f"Data types:\n{radiology_detail_df.dtypes}")
print()

## Section 4: Structure Analysis - Linkage Check

Verify if `note_id`, `subject_id`, and `hadm_id` exist in both text files and detail files. Check for one-to-one relationships.

In [ ]:
# Check common identifiers in discharge files
print("=== DISCHARGE FILES LINKAGE ===")
print(f"Discharge columns: {list(discharge_df.columns)}")
print(f"Discharge Detail columns: {list(discharge_detail_df.columns)}")
print()

# Check for common columns
discharge_common_cols = set(discharge_df.columns).intersection(set(discharge_detail_df.columns))
print(f"Common columns between discharge and discharge_detail: {discharge_common_cols}")
print()

# If note_id exists, check for one-to-one relationship
if 'note_id' in discharge_df.columns and 'note_id' in discharge_detail_df.columns:
    discharge_note_ids = set(discharge_df['note_id'])
    discharge_detail_note_ids = set(discharge_detail_df['note_id'])
    
    # Check intersection
    common_note_ids = discharge_note_ids.intersection(discharge_detail_note_ids)
    print(f"Total note_ids in discharge: {len(discharge_note_ids)}")
    print(f"Total note_ids in discharge_detail: {len(discharge_detail_note_ids)}")
    print(f"Common note_ids: {len(common_note_ids)}")
    
    # Check for duplicates
    discharge_duplicates = discharge_df['note_id'].duplicated().sum()
    detail_duplicates = discharge_detail_df['note_id'].duplicated().sum()
    print(f"Duplicate note_ids in discharge: {discharge_duplicates}")
    print(f"Duplicate note_ids in discharge_detail: {detail_duplicates}")
    
    if discharge_duplicates == 0 and detail_duplicates == 0:
        print("✓ One-to-one relationship confirmed for discharge files")
    else:
        print("⚠ Warning: Duplicate note_ids found, relationship may not be one-to-one")
print()

In [ ]:
# Check common identifiers in radiology files
print("=== RADIOLOGY FILES LINKAGE ===")
print(f"Radiology columns: {list(radiology_df.columns)}")
print(f"Radiology Detail columns: {list(radiology_detail_df.columns)}")
print()

# Check for common columns
radiology_common_cols = set(radiology_df.columns).intersection(set(radiology_detail_df.columns))
print(f"Common columns between radiology and radiology_detail: {radiology_common_cols}")
print()

# If note_id exists, check for one-to-one relationship
if 'note_id' in radiology_df.columns and 'note_id' in radiology_detail_df.columns:
    radiology_note_ids = set(radiology_df['note_id'])
    radiology_detail_note_ids = set(radiology_detail_df['note_id'])
    
    # Check intersection
    common_note_ids = radiology_note_ids.intersection(radiology_detail_note_ids)
    print(f"Total note_ids in radiology: {len(radiology_note_ids)}")
    print(f"Total note_ids in radiology_detail: {len(radiology_detail_note_ids)}")
    print(f"Common note_ids: {len(common_note_ids)}")
    
    # Check for duplicates
    radiology_duplicates = radiology_df['note_id'].duplicated().sum()
    detail_duplicates = radiology_detail_df['note_id'].duplicated().sum()
    print(f"Duplicate note_ids in radiology: {radiology_duplicates}")
    print(f"Duplicate note_ids in radiology_detail: {detail_duplicates}")
    
    if radiology_duplicates == 0 and detail_duplicates == 0:
        print("✓ One-to-one relationship confirmed for radiology files")
    else:
        print("⚠ Warning: Duplicate note_ids found, relationship may not be one-to-one")
print()

## Section 5: Metadata Inspection

Display unique values of `note_type` or `description` from the detail files to understand what kinds of reports are available.

In [ ]:
# Inspect discharge detail metadata
print("=== DISCHARGE DETAIL METADATA ===")
print(f"Sample of discharge_detail:")
print(discharge_detail_df.head())
print()

# Check for note_type or description columns
if 'note_type' in discharge_detail_df.columns:
    print(f"Unique note_type values in discharge_detail:")
    print(discharge_detail_df['note_type'].value_counts())
    print()

if 'description' in discharge_detail_df.columns:
    print(f"Unique description values in discharge_detail:")
    print(discharge_detail_df['description'].value_counts())
    print()

In [ ]:
# Inspect radiology detail metadata
print("=== RADIOLOGY DETAIL METADATA ===")
print(f"Sample of radiology_detail:")
print(radiology_detail_df.head())
print()

# Check for note_type or description columns
if 'note_type' in radiology_detail_df.columns:
    print(f"Unique note_type values in radiology_detail:")
    print(radiology_detail_df['note_type'].value_counts())
    print()

if 'description' in radiology_detail_df.columns:
    print(f"Unique description values in radiology_detail:")
    print(radiology_detail_df['description'].value_counts())
    print()

## Section 6: Text Length Analysis (Crucial for Model Selection)

Calculate word counts for each note to approximate token counts. This analysis will help determine whether we need:
- **BERT**: Standard transformer (512 tokens max)
- **Longformer/ModernBERT**: Extended context models (8k+ tokens)

In [ ]:
# Calculate word counts for discharge notes
print("=== DISCHARGE NOTES TEXT LENGTH ANALYSIS ===")

if 'text' in discharge_df.columns:
    # Calculate word count (simple split on whitespace)
    discharge_df['word_count'] = discharge_df['text'].fillna('').apply(lambda x: len(str(x).split()))
    
    # Calculate statistics
    mean_words = discharge_df['word_count'].mean()
    median_words = discharge_df['word_count'].median()
    max_words = discharge_df['word_count'].max()
    percentile_95 = discharge_df['word_count'].quantile(0.95)
    
    print(f"Word Count Statistics for Discharge Summaries:")
    print(f"  Mean: {mean_words:.0f} words")
    print(f"  Median: {median_words:.0f} words")
    print(f"  Maximum: {max_words:.0f} words")
    print(f"  95th Percentile: {percentile_95:.0f} words")
    print()
    
    # Calculate percentage exceeding BERT limit (512 tokens ≈ 512 words as rough approximation)
    bert_limit = 8192
    exceeding_bert = (discharge_df['word_count'] > bert_limit).sum()
    percent_exceeding = (exceeding_bert / len(discharge_df)) * 100
    
    print(f"Notes exceeding Modern BERT limit (8192 words): {exceeding_bert}/{len(discharge_df)} ({percent_exceeding:.1f}%)")
    print()
else:
    print("Warning: 'text' column not found in discharge_df")

In [ ]:
# Calculate word counts for radiology notes
print("=== RADIOLOGY NOTES TEXT LENGTH ANALYSIS ===")

if 'text' in radiology_df.columns:
    # Calculate word count (simple split on whitespace)
    radiology_df['word_count'] = radiology_df['text'].fillna('').apply(lambda x: len(str(x).split()))
    
    # Calculate statistics
    mean_words = radiology_df['word_count'].mean()
    median_words = radiology_df['word_count'].median()
    max_words = radiology_df['word_count'].max()
    percentile_95 = radiology_df['word_count'].quantile(0.95)
    
    print(f"Word Count Statistics for Radiology Reports:")
    print(f"  Mean: {mean_words:.0f} words")
    print(f"  Median: {median_words:.0f} words")
    print(f"  Maximum: {max_words:.0f} words")
    print(f"  95th Percentile: {percentile_95:.0f} words")
    print()
    
    # Calculate percentage exceeding BERT limit (512 tokens ≈ 512 words as rough approximation)
    bert_limit = 4096
    exceeding_bert = (radiology_df['word_count'] > bert_limit).sum()
    percent_exceeding = (exceeding_bert / len(radiology_df)) * 100
    
    print(f"Notes exceeding Modern BERT limit (8192 words): {exceeding_bert}/{len(radiology_df)} ({percent_exceeding:.1f}%)")
    print()
else:
    print("Warning: 'text' column not found in radiology_df")

## Section 7: Visualization - Word Count Histograms

Plot histograms of word counts with a vertical line at 512 words (BERT limit) to visualize how many notes would be truncated.

In [ ]:
# Create figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# BERT limit marker
bert_limit = 512

# Plot discharge notes word count histogram
if 'word_count' in discharge_df.columns:
    axes[0].hist(discharge_df['word_count'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[0].axvline(x=bert_limit, color='red', linestyle='--', linewidth=2, label=f'BERT Limit ({bert_limit} words)')
    axes[0].set_xlabel('Word Count', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    axes[0].set_title('Discharge Summaries - Word Count Distribution', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(axis='y', alpha=0.3)
else:
    axes[0].text(0.5, 0.5, 'No word_count data for discharge notes', 
                 ha='center', va='center', transform=axes[0].transAxes)

# Plot radiology notes word count histogram
if 'word_count' in radiology_df.columns:
    axes[1].hist(radiology_df['word_count'], bins=50, color='darkorange', alpha=0.7, edgecolor='black')
    axes[1].axvline(x=bert_limit, color='red', linestyle='--', linewidth=2, label=f'BERT Limit ({bert_limit} words)')
    axes[1].set_xlabel('Word Count', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)
    axes[1].set_title('Radiology Reports - Word Count Distribution', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(axis='y', alpha=0.3)
else:
    axes[1].text(0.5, 0.5, 'No word_count data for radiology notes', 
                 ha='center', va='center', transform=axes[1].transAxes)

plt.tight_layout()
plt.show()

print("Histogram visualization complete.")

## Section 8: Sample Display

Display one full example of a discharge summary and one radiology report to inspect formatting, headers, and de-identification tags.

In [ ]:
# Display a sample discharge summary
print("=" * 80)
print("SAMPLE DISCHARGE SUMMARY")
print("=" * 80)

if 'text' in discharge_df.columns and len(discharge_df) > 0:
    # Get the first non-empty text
    sample_discharge = discharge_df[discharge_df['text'].notna()].iloc[0]
    
    print(f"Note ID: {sample_discharge.get('note_id', 'N/A')}")
    print(f"Subject ID: {sample_discharge.get('subject_id', 'N/A')}")
    print(f"Hospital Admission ID: {sample_discharge.get('hadm_id', 'N/A')}")
    print(f"Word Count: {sample_discharge.get('word_count', 'N/A')}")
    print()
    print("Text Content:")
    print("-" * 80)
    print(sample_discharge['text'])
    print("-" * 80)
else:
    print("No discharge text available to display.")

print()

In [ ]:
# Display a sample radiology report
print("=" * 80)
print("SAMPLE RADIOLOGY REPORT")
print("=" * 80)

if 'text' in radiology_df.columns and len(radiology_df) > 0:
    # Get the first non-empty text
    sample_radiology = radiology_df[radiology_df['text'].notna()].iloc[0]
    
    print(f"Note ID: {sample_radiology.get('note_id', 'N/A')}")
    print(f"Subject ID: {sample_radiology.get('subject_id', 'N/A')}")
    print(f"Hospital Admission ID: {sample_radiology.get('hadm_id', 'N/A')}")
    print(f"Word Count: {sample_radiology.get('word_count', 'N/A')}")
    print()
    print("Text Content:")
    print("-" * 80)
    print(sample_radiology['text'])
    print("-" * 80)
else:
    print("No radiology text available to display.")

print()

## Generate Embeddings for Clinical Notes

In [ ]:
# Import required libraries
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import numpy as np

# Setup device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the Clinical ModernBERT model and tokenizer
model_name = "Simonlee711/Clinical_ModernBERT"
print(f"Loading model: {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, add_pooling_layer=False).to(device)

# Move model to device and set to evaluation mode
model = model.to(device)
model.eval()
print(f"Model loaded successfully and moved to {device}")

In [ ]:
def get_note_embeddings(text_list, batch_size=8):
    """
    Generate embeddings for a list of clinical notes using mean pooling.
    
    Args:
        text_list: List of text strings to embed
        batch_size: Number of texts to process in each batch
    
    Returns:
        numpy array of embeddings with shape (num_texts, embedding_dim)
    """
    all_embeddings = []
    
    # Process texts in batches
    for i in tqdm(range(0, len(text_list), batch_size), desc="Generating embeddings"):
        batch_texts = text_list[i:i + batch_size]
        
        # Tokenize the batch
        encoded_input = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=8192,
            # Clinical_ModernBERT supports 8192 token context length
            return_tensors='pt'
        )
        
        # Move input tensors to device
        encoded_input = {key: val.to(device) for key, val in encoded_input.items()}
        
        # Generate embeddings without gradient computation
        with torch.no_grad():
            model_output = model(**encoded_input)
        
        # Extract last_hidden_state for mean pooling
        # Shape: (batch_size, sequence_length, hidden_size)
        last_hidden_state = model_output.last_hidden_state
        
        # Perform mean pooling
        # Expand attention_mask to match hidden_state dimensions
        # attention_mask shape: (batch_size, sequence_length)
        # We need: (batch_size, sequence_length, hidden_size)
        attention_mask = encoded_input['attention_mask'].unsqueeze(-1).expand(last_hidden_state.size())
        
        # Sum of token embeddings (masked by attention_mask)
        sum_embeddings = torch.sum(last_hidden_state * attention_mask, dim=1)
        
        # Sum of attention mask (to get the count of real tokens)
        sum_mask = torch.clamp(attention_mask.sum(dim=1), min=1e-6)
        
        # Calculate mean by dividing sum of embeddings by sum of mask
        mean_embeddings = sum_embeddings / sum_mask
        
        # Move to CPU and convert to numpy
        batch_embeddings = mean_embeddings.cpu().numpy()
        all_embeddings.append(batch_embeddings)
    
    # Concatenate all batch embeddings
    return np.vstack(all_embeddings)

In [ ]:
# Check that discharge_df is loaded
if 'discharge_df' not in locals():
    raise ValueError("discharge_df is not loaded. Please run the data loading cells first.")

print(f"discharge_df shape: {discharge_df.shape}")
print(f"discharge_df columns: {list(discharge_df.columns)}")

# Extract text column as a list
text_list = discharge_df['text'].fillna('').tolist()
print(f"\nProcessing {len(text_list)} discharge notes...")

# Generate embeddings
embeddings = get_note_embeddings(text_list, batch_size=8)
print(f"\nEmbeddings generated with shape: {embeddings.shape}")

In [ ]:
# Create DataFrame with required columns
note_embeddings_df = pd.DataFrame({
    'note_id': discharge_df['note_id'].values,
    'subject_id': discharge_df['subject_id'].values,
    'hadm_id': discharge_df['hadm_id'].values,
    'note_embedding': list(embeddings)
})

print(f"Created note_embeddings_df with shape: {note_embeddings_df.shape}")
print(f"\nColumns: {list(note_embeddings_df.columns)}")
print(f"\nFirst few rows:")
print(note_embeddings_df.head())
print(f"\nEmbedding vector size: {len(note_embeddings_df['note_embedding'].iloc[0])}")

In [ ]:
# Save to Parquet file
output_file = 'discharge_notes_embeddings.parquet'
note_embeddings_df.to_parquet(output_file, index=False)
print(f"Saved embeddings to {output_file}")
print(f"Output DataFrame shape: {note_embeddings_df.shape}")
print(f"Single embedding vector size: {len(note_embeddings_df['note_embedding'].iloc[0])}")

# Verify by loading the saved file
print(f"\nVerifying saved file...")
loaded_df = pd.read_parquet(output_file)
print(f"Loaded DataFrame shape: {loaded_df.shape}")
print(f"Loaded DataFrame columns: {list(loaded_df.columns)}")
print(f"Loaded embedding vector size: {len(loaded_df['note_embedding'].iloc[0])}")
print(f"\nFirst few rows of loaded data:")
print(loaded_df.head())
print(f"\n✓ File saved and verified successfully!")

## Summary and Recommendations

Based on the analysis above:

1. **Model Selection**: Review the word count statistics and histogram visualizations to determine:
   - If most notes are under 512 words → Standard BERT may be sufficient
   - If many notes exceed 512 words → Consider Longformer or ModernBERT for longer context

2. **Data Linkage**: The analysis shows how `note_id`, `subject_id`, and `hadm_id` link text files to metadata files.

3. **Text Formatting**: Sample displays reveal the structure of clinical notes, including:
   - De-identification tags (e.g., `[** ... **]`)
   - Section headers
   - Formatting conventions

4. **Next Steps**:
   - Update file paths in Section 2 to your actual MIMIC-IV data location
   - Run the notebook to generate actual statistics
   - Use the results to inform P-CAFE text processing strategy